In [ ]:
import sys
import os
import time
import argparse
import numpy as np
import random
import pickle
import itertools
import copy
import pandas as pd
from scipy.special import comb
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
from tqdm.notebook import tqdm

sns.set_style("whitegrid") #darkgrid
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
SMALL_SIZE=12
MEDIUM_SIZE=15
BIGGER_SIZE=20
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow.keras.layers as layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.metrics as metrics

physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass
print(physical_devices)
import concept_model
import helper
# from test_baselines import run_eval


from utils.test_utils import arg_parser, prepare_data, get_measures
from utils.test_utils import ConceptProfiles
from utils.test_utils import get_recovered_features
from utils.ood_utils import run_ood_over_batch
from utils.stat_utils import hellinger, compute_pval, bayes_posterior, FLD, multivar_separa
from utils.plot_utils import plot_stats, plot_per_class_stats, plot_score_distr
from utils import log


In [ ]:
class ARGS:
    def __init__(self):

        self.in_data = "data/AwA2/test"
        self.out_data = "MSCOCO"
        self.workers = 4
        
        self.batch_size = 256
        self.name = "test"
        self.model = "InceptionV3"
        self.model_path = "results/AwA2/inceptionv3_AwA2_e20.weights.h5"

        self.gpu = "0"
        self.result_dir = "results/AwA2_1_baseline_s0/epoch_20"
        self.logdir = self.result_dir+"/logs"
        
        self.visualize = True
        self.visualize_with_ood = True
        self.shap = True
        self.separate = True
        self.explain = True
        self.plot = True
        self.out_data_dim = 224
        self.score = "Energy"
        self.temperature_energy = 1

        self.opt = "adam"

args = ARGS()
softmax = layers.Activation('softmax')

In [ ]:
def remove_duplicate_concepts(topic_vec, return_mapping=False):
    # Remove one concept vector if there are two vectors where the dot product is over 0.95
    # topic_vec: dim=(dim_features, n_concepts) (2048, 70)
    # print(np.shape(topic_vec))
    n_concept = topic_vec.shape[1]
    thresh = 0.95
    topic_vec_n = topic_vec/(np.linalg.norm(topic_vec,axis=0,keepdims=True)+1e-9)

    topic_vec_n_dot = np.transpose(topic_vec_n) @ topic_vec_n - np.eye(n_concept)
    dict_similar_topic = {}
    idx_delete = set()
    for i in range(n_concept):
        ith_redundant_concepts = [j for j in range(n_concept) if topic_vec_n_dot[i][j] >= 0.95]
        dict_similar_topic[i] = ith_redundant_concepts
        
        ith_redundant_concepts = [x for x in ith_redundant_concepts if x > i]
        idx_delete.update(ith_redundant_concepts)
    idx_delete = list(idx_delete)

    print(dict_similar_topic)
    print(idx_delete)

    topic_vec_r = np.delete(topic_vec, idx_delete, axis=1)


    dict_topic_mapping = {}
    count = 0
    for i in range(n_concept):
        if i in idx_delete:
            dict_topic_mapping[i] = None
        else:
            dict_topic_mapping[i] = count
            count += 1
    print('concept mapping between before/after duplicate removal......')
    print(dict_topic_mapping)
    if return_mapping:
        return topic_vec_r, dict_similar_topic, dict_topic_mapping
    else:
        return topic_vec_r, dict_similar_topic
    
def compute_concept_scores(topic_vec, feature, predict_model=None):
    # topic_vec: concept vectors (dim= (feature_dim, n_concepts))
    # feature: features extracted from an intermediate layer of trained model

    feature_n = tf.math.l2_normalize(feature, axis=3)
    topic_vec_n = tf.math.l2_normalize(topic_vec, axis=0)

    topic_prob = tf.matmul(feature_n, topic_vec_n) # K.dot

    prob_max = tf.math.reduce_max(topic_prob, axis=(1,2))
    prob_max_abs = tf.math.reduce_max(tf.abs(topic_prob), axis=(1,2))
    concept_scores = tf.where(prob_max == prob_max_abs, prob_max, -prob_max_abs)

    """
    ##for debugging
    n_concept = np.shape(concept_scores)[1]
    print(tf.reduce_mean(input_tensor=tf.nn.top_k(K.transpose(K.reshape(topic_prob,(-1,n_concept))),k=10,sorted=True).values))
    print(tf.reduce_mean(input_tensor=K.dot(K.transpose(K.variable(value=topic_vec_n)), K.variable(value=topic_vec_n)) - np.eye(n_concept)))
    """


    if predict_model: # in eager execution
        pred = softmax(predict_model(feature))
        #pred = tf.math.argmax(pred, axis=1)
        return concept_scores.numpy(), pred.numpy()
    else:
        return concept_scores

def compute_completeness(y, yhat, yhat_recov, num_class, logger=None, label=None):
    """
    compute completeness score by Yeh et al.
    :param y: groundtruth class labels, dim=(N,)
    :param yhat: predicted class labels, dim=(N,)
    :param yhat_recov: predicted class labels using recovered features, dim=(N,).
                       If label is not None, per-class predicted labels, dim=(N',) where N' <= N
    """

    acc = np.sum(y == yhat)/len(y)
    if logger:
        logger.info(f'[ID TEST] accuracy with original features: {acc}')
    
    if label is not None:
        acc_recov = np.sum(y[y==label] == yhat_recov)/len(yhat_recov)
        if logger:
            logger.info(f'[ID TEST] per-class accuracy with recovered features: {acc_recov}')
        acc_random = 1/num_class #0.5 #NOTE: check a_r = 0.5?
    else:
        acc_recov = np.sum(y == yhat_recov)/len(y)
        if logger:
            logger.info(f'[ID TEST] accuracy with recovered features: {acc_recov}')
        acc_random = 1/num_class
    
    # compute completeness
    completeness = (acc_recov - acc_random) / (acc - 1/num_class)
    if logger:
        logger.info(f'[ID TEST] completeness score: {completeness}')
    return completeness

def compute_detection_completeness(auroc, auroc_recov, logger=None):
    """
    compute detection completeness score
    """
    # compute completeness
    auroc_random = 1/2
    completeness = (auroc_recov - auroc_random) / (auroc - auroc_random)
    if logger:
        logger.info(f'[DETECTION] auroc with original features: {auroc}')
        logger.info(f'[DETECTION] auroc with recovered features: {auroc_recov}')
        logger.info(f'[DETECTION] completeness score: {completeness}')
    return completeness

def run_eval(feature_model, predict_model, in_loader, out_loader, logger, args, num_classes):
    in_scores = np.array([])
    for i, (x, y) in tqdm(enumerate(in_loader)):
        if i == len(in_loader):
            break
        score = run_ood_over_batch(x, feature_model, predict_model, args, num_classes).numpy()
        in_scores = np.concatenate([in_scores, score])
    out_scores = np.array([])
    for i, x in tqdm(enumerate(out_loader)):
        if i == len(in_loader):
            break
        score = run_ood_over_batch(x, feature_model, predict_model, args, num_classes).numpy()
        out_scores = np.concatenate([out_scores, score])
    in_examples = np.expand_dims(in_scores, axis=1)
    out_examples = np.expand_dims(out_scores, axis=1)
    auroc, aupr_in, aupr_out, fpr, thres95 = get_measures(in_examples, out_examples)
    return in_scores, out_scores, auroc, fpr, thres95

def get_class_labels(loader, savepath):
    """
    extract groundtruth class labels from data loader
    :param loader: data loader
    :param savepath: path to the numpy file
    """
    if os.path.exists(savepath):
        y = np.load(savepath)
    else:
        num_data = len(loader.filenames)
        y = []
        for (_, y_batch), _ in zip(loader, range(len(loader))):
            y.extend(y_batch)
       
        np.save(savepath, y)
    return y

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

# with tf.device('/CPU:0'):

logger = log.setup_logger(args, filename="eval_{}.log".format(args.score))
LOAD_DIR = 'data/AwA2'
TOPIC_PATH = os.path.join(args.result_dir,'topic_vec_inceptionv3.npy')
INPUT_SHAPE = (args.out_data_dim, args.out_data_dim)
TRAIN_DIR = "data/AwA2/train"
N_CLASSES = 50
N_CONCEPTS_ORIG = 100 #np.shape(topic_vec_orig)[-1]
_ = 0

if args.score == 'ODIN':
    args.batch_size = 200

if not os.path.exists(os.path.join(args.result_dir, 'plots')):
    os.makedirs(os.path.join(args.result_dir, 'plots'))
if not os.path.exists(os.path.join(args.result_dir, 'explanations')):
    os.makedirs(os.path.join(args.result_dir, 'explanations'))
if not os.path.exists(os.path.join(args.result_dir, 'explanations', args.out_data+'_'+args.score)):
    os.makedirs(os.path.join(args.result_dir, 'explanations', args.out_data+'_'+args.score))
explain_dir = os.path.join(args.result_dir, 'explanations', args.out_data+'_'+args.score)

in_loader, out_loader = prepare_data(args, logger)

## load trained_model
logger.info(f"Loading model from {args.model_path}")
feature_model, predict_model = helper.load_model_inception_new(_, in_loader, batch_size=args.batch_size, 
                                    input_size=INPUT_SHAPE, pretrain=True, modelname=args.model_path)

in_test_features = feature_model.predict(in_loader)
N_IN = in_test_features.shape[0]
# out_test_features = feature_model.predict(out_loader, steps=len(in_loader))
# N_OUT = out_test_features.shape[0]


In [ ]:
## load topic model
topic_model = concept_model.TopicModel(in_test_features, N_CONCEPTS_ORIG, thres=0.0, predict=predict_model)
topic_model(in_test_features)
topic_model.load_weights(os.path.dirname(args.result_dir)+'/topic_epoch20.weights.h5')
for layer in topic_model.layers:
    layer.trainable = False


topic_vec_orig = topic_model.layers[0].get_weights()[0]
np.save(args.result_dir+'/topic_vec_orig.npy', topic_vec_orig)
logger.info(f'Number of concepts before removing duplicate ones: {str(N_CONCEPTS_ORIG)}')

topic_vec, dict_dupl_topic = remove_duplicate_concepts(topic_vec_orig)
N_CONCEPTS = np.shape(topic_vec)[-1] # 25
logger.info(f'Number of concepts after removing duplicate ones: {str(N_CONCEPTS)}')

in_test_concepts, in_test_logits = compute_concept_scores(topic_vec, in_test_features, predict_model)
in_test_yhat = np.argmax(in_test_logits, axis=1) 

# out_test_concepts, out_test_logits = compute_concept_scores(topic_vec, out_test_features, predict_model)
# out_test_yhat = np.argmax(out_test_logits, axis=1)

In [ ]:
# target OOD detector
logger.info("[ID TEST] performance of target OOD detector with test set...")

## Evaluating the difference between two worlds......
y_test = np.argmax(np.load('data/AwA2/y_test.npy'), axis=1) # true labels

# compute completeness scores
_, logits_recov, _ = topic_model(in_test_features)
in_test_yhat_recov = tf.math.argmax(logits_recov, axis=1).numpy()
compute_completeness(y_test, in_test_yhat, in_test_yhat_recov, N_CLASSES, logger)

# in_test_scores, out_test_scores, auroc, fpr, thres95 = run_eval(feature_model, predict_model, in_loader, out_loader, logger, args, N_CLASSES)
# in_test_scores, out_test_scores, thres95, auroc = np.random.rand(N_IN), np.random.rand(N_OUT), 0.5419758558273315, 0.955332290562036
# compute_detection_completeness(auroc, auroc_recov, logger)


In [ ]:
### Compute completeness
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

TRAIN_DIR = "data/AwA2/train"
VAL_DIR = "data/AwA2/val"

print('Loading images through generators ...')
datagen = ImageDataGenerator(rescale=1. / 255.)
train_loader = datagen.flow_from_directory(TRAIN_DIR,
                                            batch_size=args.batch_size,
                                            target_size=(224, 224),
                                            class_mode='categorical',
                                            shuffle=True)
val_loader = datagen.flow_from_directory(VAL_DIR,
                                        batch_size=args.batch_size,
                                        target_size=(224, 224),
                                        class_mode='categorical',
                                        shuffle=False)

y_val_ = np.load('data/AwA2/y_val.npy')
y_test_ = np.load('data/AwA2/y_test.npy')

if args.opt =='sgd':
    """
    optimizer = SGD(lr=0.1)
    optimizer_state = [optimizer.iterations, optimizer.lr, optimizer.momentum, optimizer.decay]
    optimizer_reset = tf.compat.v1.variables_initializer(optimizer_state)
    """
    optimizer = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
elif args.opt =='adam':
    optimizer = Adam(learning_rate=0.01)
    optimizer_state = [optimizer.iterations, optimizer.learning_rate, optimizer.beta_1, optimizer.beta_2, optimizer.weight_decay]
    optimizer_reset = tf.compat.v1.variables_initializer(optimizer_state)

train_acc_metric = metrics.CategoricalAccuracy()
val_acc_metric = metrics.CategoricalAccuracy()
test_acc_metric = metrics.CategoricalAccuracy()
softmax = layers.Activation('softmax')

for layer in topic_model.layers:
    layer.trainable = True
topic_model.layers[1].trainable = False
topic_model.layers[-1].trainable = False

@tf.function
def train_step(x_in, y_in, x_out=None, thres=None):
    #tf.keras.applications.inception_v3.preprocess_input(x_in)
    f_in = feature_model(x_in)

    obj_terms = {} # terms in the objective function
    with tf.GradientTape() as tape:
        f_in_recov, logits_in, topic_vec_n = topic_model(f_in, training=True)
        pred_in = softmax(logits_in) # class prediction using concept scores

        # total loss
        CE_IN = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_in, pred_in))
        loss = CE_IN
        obj_terms['[ID] CE'] = CE_IN
    
    train_acc_metric.update_state(y_in, logits_in)
    print(obj_terms)

    # calculate the gradients using our tape and then update the model weights
    print(topic_model.trainable_variables)
    grads = tape.gradient(loss, topic_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, topic_model.trainable_variables))
    #input()
    return obj_terms

os.makedirs(args.logdir, exist_ok=True)
df_obj_terms = pd.DataFrame()
for epoch in range(20):
    logger.info(f"\n[INFO] starting epoch {epoch}/{20} ---------------------------------")
    sys.stdout.flush()
    epochStart = time.time()
    
    for step, (x_in, y_in) in enumerate(train_loader):
        
        step += 1 # starts from 1
        if step > len(train_loader):
            break
        obj_terms = train_step(x_in, y_in)

        # Log every 50 batches
        if step % 20 == 0:
            #print(topic_model.layers[0].get_weights()[0])
            for term in obj_terms:
                logger.info(f'[STEP{step}] {term}: {obj_terms[term]}')
        for term in obj_terms:
            obj_terms[term] = obj_terms[term].numpy()
        obj_terms["epoch"] = epoch
        obj_terms["step"] = step
        df_obj = pd.Series(obj_terms)
        df_obj_terms = pd.concat([df_obj_terms, pd.DataFrame(df_obj).T], axis=0)
    
    train_acc = train_acc_metric.result()
    logger.info("Training acc over epoch: %.4f" % (float(train_acc),))
    
    # show timing information for the epoch
    epochEnd = time.time()
    elapsed = (epochEnd - epochStart) / 60.0
    logger.info("Time taken: %.2f minutes" % (elapsed))

    df_obj_terms = df_obj_terms.reset_index(drop=True)
    df_obj_terms_melt = pd.melt(df_obj_terms, id_vars=["epoch", "step"], 
                                value_vars=[col for col in df_obj_terms.columns if col in 
                                            ['[ID] CE']],
                                var_name="loss_term", value_name="loss_value")

    plt.figure()
    sns.lineplot(data=df_obj_terms_melt, x="epoch", y="loss_value", hue="loss_term")
    plt.savefig(args.logdir+"/model_compl_finetune_loss.png")
    plt.close()

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_state()
    topic_model.save_weights(os.path.join(args.logdir, args.name,'topic_compl_finetune_epoch{}.weights.h5'.format(epoch)))

    _, logits_val, _ = topic_model(feature_model.predict(val_loader), training=False)
    val_acc_metric.update_state(y_val_, logits_val)
    val_acc = val_acc_metric.result()
    logger.info("[EPOCH %d] Validation acc: %.4f" % (epoch, float(val_acc)))
    val_acc_metric.reset_state()
    del logits_val

    _, logits_test, _ = topic_model(feature_model.predict(in_loader), training=False)
    test_acc_metric.update_state(y_test_, logits_test)
    test_acc = test_acc_metric.result()
    logger.info("[EPOCH %d] Test acc: %.4f" % (epoch, float(test_acc)))
    test_acc_metric.reset_state()
    del logits_test

    logger.flush()

for layer in topic_model.layers:
    layer.trainable = False

In [ ]:
# compute completeness scores
_, logits_recov, _ = topic_model(in_test_features)
in_test_yhat_recov = tf.math.argmax(logits_recov, axis=1).numpy()
compute_completeness(y_test, in_test_yhat, in_test_yhat_recov, N_CLASSES, logger)
# compute_detection_completeness(auroc, auroc_recov, logger)